In [4]:
# import some common packages for data analysis and visualizations
import sys
import numpy as np
import scipy as sp
import scipy.interpolate as spint
import matplotlib as mpl
import matplotlib.pyplot as plt

# import some code snippets for convenience
# install this package via `pip install -U pyridoxine`
import pyridoxine.plt as rxplt

# needed for loadding & processing athena++ data
import h5py
# run this file to load the class I wrote for easier handling of athena++ data: Raw_Data_Restricted
%run ./rixin_rw.py  # which also requires "athena_read.py" in the same folder


In [5]:
#jupyter magic command that sets the matplotlib backend
#nbagg = "notbook aggregation backend"
%matplotlib nbagg
np.set_printoptions(edgeitems=5, linewidth=200, precision=5)  # quick default setup, handy for debugging and inspecting simulation data


In [6]:
# import more python modules for fancy visualizations
import plotly.graph_objects as plygo
import magpylib as magpy #magnetic field visualizion and simulation
import pyvista as pv #3D volumetric and mesh-based visualization
pv.set_jupyter_backend('trame') #enables interactive 3D rendering in Jupyter


# 1. Load File

In [7]:
# make sure the data file is in the same folder as this Jupyter Notebook
# we now load the last snapshot of the simulation "3D Model B" from Huang, Yu, Lee, Dong & Bai 2025
# Raw_Data_Restricted is a class from rixin_rw.py
ds = Raw_Data_Restricted("/home/dasein/repositories/notebooks_julian/disk_planet_3D.cons.00400.athdf")


In [8]:
# For simulation "3D Model B", we have the following parameters
#
q = 3e-4   # Planet is only 0.03% of the star's mass
α = 1e-2

# derived physical parameters
R_H = (q/3)**(1/3)  # Hill radius (without a so more like fraction)
R_Bondi = q/0.05**2  # Bondi radius  0.05 = ratio of dics thickness = soundspeed/keplerian velocity (orbital)
print(R_H)

# planet position in spherical and cartesian coordinates
planet_pos = np.array([1.0, np.pi/2, np.pi])
planet_pos_car = np.array([-1.0, 0, 0])


0.046415888336127795


In [ ]:
# (Optional) Interactive Phi Angle
#phiangle = float(input("Choose your ϕ angles [0, 2π)"))

for phiangle in range(0, 360):
    rxplt.plt_params('paper')
    fig = plt.figure(figsize=(11,8.5))

    # Polar axes: [left, bottom, width, height]
    ax = fig.add_axes([0.2, 0.1, 0.7, 1.0], projection='polar')

    # Map degree phiangle -> discrete index along ccphi
    #i_phi = int(np.round(phiangle / 720.0 * ds.ccphi.size)) % ds.ccphi.size

    # Overplot levels 1 and 2
    for _lev in range (1, 3):
        ds.get_level(_lev)

        # extract dust density at given phi (where the planet locates)
        Q = ds['rhod_1'][int(ds.ccphi.size*phiangle//360), :, :]
        #Q = ds['rhod_1'][i_phi, :, :]
        im = ax.pcolormesh(
            ds.cctheta,
            ds.ccr,
            np.log10(Q).T,
            shading='auto',
            cmap="plasma",
            vmin=-8, vmax=0.1,
            rasterized=True
        )

    # set theta to increase in the clockwise direction
    ax.set_theta_direction(-1)
    # position θ=0 at North and draw a faint white grid
    ax.set_theta_zero_location('N')
    ax.grid(True, ls=':', alpha=0.3, c='w')

    # limit θ to ±5 scale heights around the midplane (in degrees)
    H = 0.05
    ax.set_thetamin(180/np.pi * (np.pi/2 - 5*H))
    ax.set_thetamax(180/np.pi * (np.pi/2 + 5*H))

    # set angular ticks at ±3H and 0, and label them accordingly
    ax.set_xticks([np.pi/2 - 3*H, np.pi/2, np.pi/2 + 3*H])
    ax.set_xticklabels([r"$3H$", r"$0$", r"$-3H$"])
    # pad the angular tick labels outward for readability
    ax.tick_params(axis='x', which='major', pad=24)

    # reduce radial ticks to only 0.5 and 1 for clarity
    ax.set_rticks([0.5, 1])

    # annotate the plot with φ location in blue, centered at given figure coordinates
    # fig.text(0.195, 0.645, fr"$\phi={phiangle:.2f}$", ha='center', va='center',
    fig.text(0.195, 0.645, fr"$\phi={phiangle:03d}^\circ$", ha='center', va='center',
             fontsize=30, color='blue')

    # add a new horizontal axes for the colorbar
    cax = fig.add_axes([0.125, 0.9, 0.75, 0.02])
    cbar = fig.colorbar(im, cax=cax, orientation='horizontal')
    # move the colorbar label to the top of the bar
    cbar.ax.xaxis.set_label_position('top')
    # set the colorbar label including simulation time and log dust density
    cbar.set_label(
        # fr"3D Model B, $t={ds.t/(2*np.pi):.1f}"+r"P_{\rm orb}$, " + r"$\log \rho_{\rm d}$", labelpad=12)
        fr"3D Model B, $t={ds.t/(2*np.pi):.1f}P_{{\rm orb}}$, $\log \rho_{{\rm d}}$",
        labelpad=12
    )

    filename = f"/home/dasein/repositories/notebooks_julian/disk_slices/plot_{phiangle:03d}.png"
    fig.savefig(filename)
    plt.close(fig)
